# 🔍 Kvasir-SEG U-Net Demo

This notebook demonstrates how to download the Kvasir-SEG dataset, define a U-Net segmentation model, and perform inference using a pretrained model.

📌 If you haven't already, make sure the file structure from the GitHub repo is accessible or clone the repo.

In [1]:
# Clone the repo (if not already cloned)
!git clone https://github.com/GaurangTari4/KvasirSeg-Unet.git
%cd KvasirSeg-Unet

In [2]:
!pip install -r requirements.txt

In [3]:
from dataset.download import download_and_extract_kvasir

download_and_extract_kvasir()

In [4]:
import torch
from model.unet import UNet
from dataset.loader import load_dataset, KvasirSegDataset
from torch.utils.data import DataLoader
import os

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load dataset
images, masks = load_dataset("kvasir-seg/images", "kvasir-seg/masks")
dataset = KvasirSegDataset(images, masks, transform=None)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

# Load model
model = UNet(in_channels=3, out_channels=1).to(DEVICE)
model.load_state_dict(torch.load("models/unet_kvasirseg.pth", map_location=DEVICE))
model.eval()

In [5]:
from utils.visualize import plot_prediction

inputs, masks = next(iter(loader))
inputs, masks = inputs.to(DEVICE), masks.to(DEVICE)
with torch.no_grad():
    outputs = torch.sigmoid(model(inputs))

plot_prediction(inputs.cpu(), masks.cpu(), outputs.cpu())

### ✅ Conclusion

That's it! You can now test different images, fine-tune the model, or run full evaluations.